In [1]:
import os
import sys

try:
    from google.colab import drive
    IN_COLAB = True
except ModuleNotFoundError as e:
    IN_COLAB = False

if IN_COLAB:
    drive.mount('/content/drive', force_remount=True)
    MY_DRIVE_PATH ='/content/drive/MyDrive/Technion/Diffusion-AD-Project/'
    sys.path.append(MY_DRIVE_PATH)
    os.chdir(MY_DRIVE_PATH)
    print(f"Current working directory at {os.getcwd()}")


In [4]:
if IN_COLAB:
    !pip install blobfile &> /dev/null
    !pip install mpi4py &> /dev/null
    !pip install pytorch-lightning &> /dev/null


In [ ]:
import pytorch_lightning as pl
from IPython.display import Markdown as md
from utils.models import ModelLoader
from utils.noiser import *
from utils.denoiser import *
from utils.error_map import *
from utils.anomaly_scorer import *
from utils.data_frame_manager import DataFrameManager
from core.diffusion_ad import DiffusionAD
from extern.guided_diffusion.guided_diffusion.unet import UNetModel
from config.configuration import DIFFUSION_AD_HPARAMS, DIFFUSION_AD_REQUIRED_HPARAMS, \
    CATEGORY_TO_NOISE_TIMESTEPS, CATEGORY_TO_TYPE, CATEGORY_TYPE_OBJECT, CATEGORY_TYPE_TEXTURE

In [ ]:
model: UNetModel = None
torch.set_float32_matmul_precision('medium')

# **<u>Initialize Experiment</u>**

First, we load and construct all the needed objects for our experiment.

In [ ]:
if not model:
    model, diffusion = ModelLoader().get_model('256x256_uncond')

In [ ]:
noiser = TimestepUniformNoiser(diffusion)
denoiser = ModelTimestepUniformDenoiser(model, diffusion)
anomaly_map_generator = BatchFilteredSquaredError()
anomaly_scorer = MaxValueAnomalyScorer()

In [ ]:
diffusion_ad = DiffusionAD(noiser,
                           denoiser,
                           anomaly_map_generator,
                           anomaly_scorer,
                           DIFFUSION_AD_HPARAMS)
diffusion_ad.args.verbosity = 2

In [ ]:
trainer = {
    category: pl.Trainer.from_argparse_args(diffusion_ad.args, 
                                            default_root_dir=os.path.join(diffusion_ad.args.root_output_dir, category), 
                                            max_epochs=diffusion_ad.args.num_epochs,
                                            accelerator='gpu',
                                            devices=1) \
           for category in CATEGORY_TO_NOISE_TIMESTEPS.keys()
}

# **<u>Run Experiment</u>**

## **<u>Object Categories</u>**

First, we will run the evaluation on the different object categories.

In [ ]:
OBJECT_CATEGORIES = [
    category for category, category_type in CATEGORY_TO_TYPE.items() 
        if category_type == CATEGORY_TYPE_OBJECT
]

md(f'**<u>The object categories are:</u>**\n\n{OBJECT_CATEGORIES}')

In [ ]:
remaining_categories = diffusion_ad.get_remaining_categories()
overwrite = False
print(f'Remaining categories: {set(OBJECT_CATEGORIES) & remaining_categories}\n'
      f'Overwrite: {overwrite}')

for category in OBJECT_CATEGORIES:
    if overwrite or category in remaining_categories:
        print('\033[4m\033[1m' + f'Evaluating category: {category}' + '\033[0m')
        diffusion_ad.args.category = category
        try:
            trainer[category].test(diffusion_ad)
        
        except FileNotFoundError as e:
            print(e, f'\nFiles missing for category {category}! Please fix your dataset folder.\n')

## **<u>Texture Categories</u>**

Now, we will run the evaluation on the different texture categories.

In [ ]:
TEXTURE_CATEGORIES = [
    category for category, category_type in CATEGORY_TO_TYPE.items() 
        if category_type == CATEGORY_TYPE_TEXTURE
]

md(f'**<u>The texture categories are:</u>**\n\n{TEXTURE_CATEGORIES}')

In [ ]:
remaining_categories = diffusion_ad.get_remaining_categories()
overwrite = False
print(f'Remaining categories: {set(TEXTURE_CATEGORIES) & remaining_categories}\n'
      f'Overwrite: {overwrite}')

for category in TEXTURE_CATEGORIES:
    if overwrite or category in remaining_categories:
        print('\033[4m\033[1m' + f'Evaluating category: {category}' + '\033[0m')
        diffusion_ad.args.category = category
        try:
            trainer[category].test(diffusion_ad)
        
        except FileNotFoundError as e:
            print(e, f'\nFiles missing for category {category}! Please fix your dataset folder.\n')